# HW3 GEE_Whiz: Dataset Overview and Use Case Examples
## EDS 220, Fall 2021

## Mangrove Canopy Coverage from LANDSAT 8 Before and After Restoration

## Authors
- Allie Cole, UC Santa Barbara (icole@bren.ucsb.edu) <br>
  https://alliecole.github.io/
- Clarissa Boyajia, UC Santa Barbara (cboyajian@bren.ucsb.edu) <br>
  https://cboyajian.github.io/
- Scout Leonard, UC Santa Barbara (scout@bren.ucsb.edu) <br>
  https://scoutcleonard.github.io/

## Table of Contents

[1. Purpose](#purpose)

[2. Dataset Description](#overview)

[3. Data I/O](#io)

[4. Metadata Display and Basic Visualization](#display)

[5. Use Case Examples](#usecases)

[6. Future Analysis](#future)

[7. References](#references)

<a id='purpose'></a> 
### Notebook Purpose

This notebook was created to give an overview and short tutorial on how to calculate mangrove canopy coverage percentages using data from the NASA & USGS Landsat 8 dataset. For the tutorial, we will focus on the Cacheu River Mangroves Natural Park in the Republic of Guinea-Bissau in West Africa where restoration efforts began in 2017.

<a id='overview'></a> 
### Dataset Description

[The Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/landsat-8) data collection is supported jointly by NASA and hosted by the USGS. It is accessible through the [Google Earth Engine (GEE) API](https://earthengine.google.com). Landsat 8 data can be downloaded in many different formats depending on how you want to manipulate it. The data includes 9 bands: 5 visible and near-infared (NIR) bands, 2 short-wave infared (SWIR), and 2 thermal infared (TIR) bands. 

Various products offer varying combinations of bands. The data found in these products comes from USGS in the form of .TIFFs and .JPEGs.

The Landsat 8 satellite data were collected as global atmospherically corrected surface reflectance from the Landsat 8 OLI/TIRS sensors for every 8-16 days starting in 2013. The downside to this data is the lower resolution, especially for the context of visualizing mangroves, which we discuss this more in our calculations section. 


<a id='io'></a> 
### Dataset Input/Output 

Below we install necessary packages, authenticate GEE, define parameters for our mangrove calculations, and explore the data by pulling a sample image, associated variables, and creating a test map to see that the visible bands work

In [1]:
#import all necessary packages 
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
#authorize google earth engine access 
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWiH41JUDpKvQD_RON98BgaI3OJOU6r6tA7jO6AJ3up-vzeMYHEixHw



Successfully saved authorization token.


In [6]:
#set parameters
gee_landsat8_api = "LANDSAT/LC08/C01/T1_TOA"

gb_lat = 12.1582165861 #latitude for poi in guinea-bissau
gb_lon = -16.283462202 #latitude for poi in guinea-bissau

scale = 1000   # scale in m

In [7]:
#load data
gdat = ee.ImageCollection(gee_landsat8_api)

<a id='display'></a> 
### Metadata Display and Basic Visualization

#### Landsat8 Data quality and missing data

Landsat 8 revisits a location every 8-16 days, which is at a much higher rate when compared to most other satellites. Additionally, Landsat 8 captures a range of multispectral bands, which we explore below. The Landsat 8 resolution is 30 meters (visible, NIR, SWIR); 100 meters (thermal); and 15 meters (panchromatic).

The frequent revisitation rate and free access makes this a good dataset for our purposes. The downside to this data is the lower resolution. 

#### Exploring the metadata

To explore the Landsat 8 metadata, we call the first image in the image collaction below, and then extract the variable names from that image. We also visualized in the test map below the Cacheu National Park Mangrove Forest in Guinea-Bissau.

In [20]:
# pull the first image in the collection 


In [8]:
# Looking at the Meta Data properties 
    # ee.List of metadata properties

The code chunk below uses the first image in the collection to call the variables included in the images of our Landsat 8 data:

In [22]:
#extract a list containing the names of the bands 


"['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']"

We can see from the returned results that the dataset has 12 variables. Their names are:

- `B1` : coastal aerosol
- `B2` : blue
- `B3`: green
- `B4`: red
- `B5` : near infared
- `B6` : shortwave infared 1
- `B7` : shortwave infared 2
- `B8` : band 8 panochromatic
- `B9`: cirrus
- `B10` : thermal infared 1
- `B11` : thermal infared 2
- `sr_aerosol`: aerosol attricutes
- `pixel_qa` : pixel quality attributes generated from the CFMASK algorithm 
- `radsat_qa`: radiometric saturation QA

We know the definitions of the bands from [Landsat 8 metadata](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR) from Google Earth Engine.

In [23]:
#select the region: Cacheu National Park Mangrove Forest


In [24]:
#explore a data feature of the Landsat images; we chose Band 2


In [25]:
#create a dataframe using Band 2 measurements for Sundarabans mangrove forest over time 


                        0          1          2              3         4
0                      id  longitude   latitude           time        B2
1    LC08_204052_20130503 -16.281965  12.158697  1367580233310  0.119386
2    LC08_204052_20130519 -16.281965  12.158697  1368962646080  0.115767
3    LC08_204052_20130706 -16.281965  12.158697  1373109844400  0.546247
4    LC08_204052_20130823 -16.281965  12.158697  1377257047690  0.337861
..                    ...        ...        ...            ...       ...
465  LC08_205052_20210430 -16.281965  12.158697  1619782069053  0.121051
466  LC08_205052_20210516 -16.281965  12.158697  1621164471929  0.127367
467  LC08_205052_20210617 -16.281965  12.158697  1623929286840  0.119166
468  LC08_205052_20211007 -16.281965  12.158697  1633606119659  0.157453
469  LC08_205052_20211108 -16.281965  12.158697  1636370919687  0.108191

[470 rows x 5 columns]


Basic Visualization: 

Below, we look at our dataset using a visualization. We create a simple map of our point of interest with bands B4, B3, and B2. 

In [26]:
#Creating a base map using our gb point


In [27]:
# Filtering using the point


In [28]:
# selecting the bands for our paramaters 


In [29]:
#testing the map 


Map(center=[12.1582165861, -16.283462202], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [30]:
#adding our visable layers 


<a id='usecases'></a> 
### Use Case Examples

The Cacheu National Park Mangrove Forest in Guinea-Bissau is a park located on the Cacheu River. The forest is the largest compact mangrove envrionment on West Africa's coast. The forest has been the recipient of reforestation efforts beginning in 2017. Knowing this documented effort overlapped with Landsat 8 data inspired our group to investigate vegetation indices in order to calculate canopy coverage before and after restoration interventions. 

This analysis looks at the success of this specific restoration effort by visualizing vegetation indices in 2014 and 2019. This analysis will be useful for stakeholders in mangrove restoration and/or management projects. Understanding and measuring the health and growth of mangrove forests is important because of their storm-protection benefits to global coastlines. 

The code below utilizes the multispectral bands from Landsat 8 images over Cacheu National Park Mangrove Forest to calculate various vegetation indices for the mangrove forest. The Landsat 8 data for the visual and infared bands is lower resolution, with pixels at 30 meters x 30 meters. Because vegetation indices assume these 30 m x 30 m units are homogenous, the utilization of multiple indices can help to better measure mangrove canopy coverage accuartely. This is because 30 x 30 m areas have heterogenous coverage, including foliage, water, sand, and soil. 

We calculate multiple indices inspired by a 2017 paper published in Elsevier, titled "A new approach for calculating mangrive camopy cover using Landsat 8 imagery," by Hesham Abd-El Monsef and Scot E. Smith. The authors calculate 7 indices to investigate mangrove canopy coverage. In addition to calculating the Normalized Differnece Vegetation Index (NDVI), we also calculate Normalized Difference Infared Index (NDII), and Normalized Difference Built-up Index (NDBI). 

The reason we utilized these indices is as follows: 

- **NDVI:** a standard measure of vegetation health that measures cholorphyll 
- **NDII:** a measure that's sensitive to changes in water storage and plant biomass
- **NDBI:** a measure used to distinguish bare soil from vegetation 

First we'll filter for the area of interest and create NDVI for 2014 and 2019, befpre and after restoration efforts. 

$$ NDVI = \frac{NIR - red}{NIR + red} $$

In [43]:
#filter image collection using our point in Chacheu National Park Mangrove Forest


In [44]:
# create function to calculate NDVI for a given input image



# apply fuction to all images in our point of interest filter


#create visable parameters for each band used in NDVI calculations


In [45]:
#filter out the cloudy images over 20% cloud cover


#filter for the time period 2014 

#creating the basemap 


#add 2014 NDVI layer to map


Next, we create a second layer. This one represents NDVI for 2019. 

In [46]:
#filter for the time period 2019 and create a temporal average 


#add 2019 NDVI layer to map


In [47]:
#calling the NDVI map 


Map(center=[12.1582165861, -16.283462202], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

Next, we'll calculate NDII layers for the same two years:

$$NDII = \frac{NIR_{B5} - MidIR{_{B6}}}{NIR_{B5} + MidIR{_{B6}}}$$

In [ ]:
# create function to calculate NDII for a given input image
def addNDII(image):
    mir = image.select('B6')
    nir = image.select('B5')
    
    ndii = (nir.subtract(mir)).divide((nir.add(mir))).rename('NDII')
    
    return image.addBands(ndii)

# apply fuction to all images in our point of interest filter
gdat_withndii = gdat_filt.map(addNDII) 

#create visable parameters for each band used in NDII calculations
ndiiParams = {'bands': 'NDII',
              'min': -1, 
              'max': 1, 
              'palette': ['blue', 'white', 'green']
             }

#filter out the cloudy images over 20% cloud cover
dat_nocld_ndii = gdat_withndii.filter('CLOUD_COVER < 20')

#filter for the time period 2014 and create a temporal average 
date_2014_ndii = dat_nocld_ndii.filter(ee.Filter.date('2014-01-01', '2014-12-31')).mean();

#creating the basemap 
map_ndii = geemap.Map(center = [gb_lat, gb_lon], zoom = 12)

#add 2014 NDII layer to map
map_ndii.addLayer(date_2014_ndii, ndiiParams, "2014")

#filter for the time period 2019 and create a temporal average 
date_2019_ndii = dat_nocld_ndii.filter(ee.Filter.date('2019-01-01', '2019-12-31')).mean();

#add 2019 NDII layer to map
map_ndii.addLayer(date_2019_ndii, ndiiParams, "2019")

In [35]:
#calling the NDII map
map_ndii

Map(center=[12.1582165861, -16.283462202], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

Next, we'll calculate NDBI layers for the same two years:

$$NDBI = \frac{MidIR_{B6} - NIR}{MidIR_{B6} + NIR}$$

In [36]:
# create function to calculate NDBI for a given input image
def addNDBI(image):
    mir = image.select('B6')
    nir = image.select('B5')
    
    ndbi = (mir.subtract(nir)).divide((mir.add(nir))).rename('NDBI')
    
    return image.addBands(ndbi)

# apply fuction to all images in our point of interest filter
gdat_withndbi = gdat_filt.map(addNDBI) 

#create visable parameters for each band used in NDBI calculations
ndbiParams = {'bands': 'NDBI',
              'min': -1, 
              'max': 1, 
              'palette': ['blue', 'white', 'green']
             }

#filter out the cloudy images over 20% cloud cover
dat_nocld_ndbi = gdat_withndbi.filter('CLOUD_COVER < 20')

#filter for the time period 2014 and create a temporal average 
date_2014_ndbi = dat_nocld_ndbi.filter(ee.Filter.date('2014-01-01', '2014-12-31')).mean();

#creating the basemap 
map_ndbi = geemap.Map(center = [gb_lat, gb_lon], zoom = 12)

#add 2014 NDBI layer to map
map_ndbi.addLayer(date_2014_ndbi, ndbiParams, "2014")

#filter for the time period 2019 and create a temporal average 
date_2019_ndbi = dat_nocld_ndbi.filter(ee.Filter.date('2019-01-01', '2019-12-31')).mean();

#add 2019 NDBI layer to map
map_ndbi.addLayer(date_2019_ndbi, ndbiParams, "2019")

In [37]:
#calling the NDBI map
map_ndbi

Map(center=[12.1582165861, -16.283462202], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

<a id='future'></a> 
### Future analysis

This project can be taken a step further by turning each image created into a rate of mangrove percent cover, and comparing layers before and after restoration. Mangrove planting often occurs in areas that are hard to survey, where recording accute and consistant coverage can be difficult. Using this method to calculate percent cover is both cost and time effective as no travel is nessisary and the images are free. Tracking mangorve percent cover can have large implications for both human and ecosystem health. It has been shown (reference) that mangroves protect the coastline from storm damage and storm surges, events that can cost millions of dollars in damages and take human lives. Healthy mangrove forests also protect the coast line from erosion and provide important habitat and nursery grounds for many species. 

By incorporating a multi-index method of calculating percent cover, nonprofits and orginazations can easily track the growth and decline of mangrove ecosystems and set clear deliverables for restoration projects. This method not only makes tracking growth much less resource intensive, but also enables more frequent growth tracking as it does not require field data collection, which can be sporadic.

<a id='references'></a> 
### References

1. "A new approach for estimating mangrove canopy cover using Landsat 8 ": https://www.sciencedirect.com/science/article/pii/S0168169916303490
2. "Conserving Biodiversity of the Cacheu Mangroves National Park at Guinea Bissau": https://www.wetlands.org/publications/conserving-biodiversity-cacheu-mangroves-national-park-guinea-bissau/
3. "Compare NDVI extracted from Landsat 8 imagery with that from Landsat 7 imagery" http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.949.9781&rep=rep1&type=pdf 
